In [ ]:
# 1. Imports e configurações
import os
import pandas as pd
pd.options.display.max_columns = 120
pd.options.display.width = 180
import warnings 
from dateutil import parser
from typing import Iterable, Optional
import numpy as np

warnings.filterwarnings('ignore')

In [ ]:
# 2b. Leitura e concatenação dos arquivos (com dtype=str para evitar inferência incorreta)
folder_path = r"C:\Users\u8178\OneDrive - GRUPO EQUATORIAL ENERGIA\99 - QUASAR\Uber\parent"
csv_paths = []
for fn in os.listdir(folder_path):
    p = os.path.join(folder_path, fn)
    if os.path.isfile(p) and fn.lower().endswith(('.csv','.xlsx','.xls')):
        csv_paths.append(p)
all_dfs = []
for path in csv_paths:
    if path.lower().endswith('.csv'):
        tmp = pd.read_csv(path, header=4, sep=';', dtype=str)
    else:
        tmp = pd.read_excel(path, header=5, dtype=str)
    all_dfs.append(tmp)
df = pd.concat(all_dfs, ignore_index=True)

df.head()

In [ ]:
# 3b. Dicionário de renomeação (mantido como referência)
new_column_names = {
    'UUID da organização': 'uuid_organizacao',
    'Nome da organização': 'nome_organizacao',
    'Moeda local': 'moeda_local',
    'ID da viagem/Uber Eats': 'id_viagem',
    'Registro de data e hora da transação (UTC)': 'data_hora_transacao_utc',
    'Data da solicitação (UTC)': 'data_solicitacao_utc',
    'Hora da solicitação (UTC)': 'hora_solicitacao_utc',
    'Data da solicitação (local)': 'data_solicitacao_local',
    'Hora da solicitação (local)': 'hora_solicitacao_local',
    'Data de chegada (UTC)': 'data_chegada_utc',
    'Hora de chegada (UTC)': 'hora_chegada_utc',
    'Data de chegada (local)': 'data_chegada_local',
    'Hora de chegada (local)': 'hora_chegada_local',
    'Compensação do fuso horário de solicitação a partir do UTC': 'comp_fuso_horario_utc',
    'Nome': 'nome',
    'Sobrenome': 'sobrenome',
    'E-mail': 'email',
    'ID do funcionário': 'id_funcionario',
    'Serviço': 'servico',
    'Cidade': 'cidade',
    'Distância (mi)': 'distancia_mi',
    'Duração (min)': 'duracao_min',
    'Endereço de partida': 'endereco_partida',
    'Endereço de destino': 'endereco_destino',
    'Código da despesa': 'codigo_despesa',
    'Detalhamento da despesa': 'detalhamento_despesa',
    'Faturas': 'faturas',
    'Programa': 'programa',
    'Grupo': 'grupo',
    'Forma de pagamento': 'forma_pagamento',
    'Tipo de transação': 'tipo_transacao',
    'Valor na moeda local (sem tributos)': 'valor_local_sem_tributos',
    'Tributos na moeda local': 'tributos_local',
    'Valor extra em moeda local': 'valor_extra_local',
    'Valor da transação na moeda local (com tributos)': 'valor_transacao_local_com_tributos',
    'Código da moeda local': 'codigo_moeda_local',
    'Valor na moeda local (sem impostos)': 'valor_local_sem_impostos',
    'Impostos na moeda local': 'impostos_local',
    'Valor extra na moeda local': 'valor_extra_local_dup',
    'Valor da transação na moeda local (com impostos)': 'valor_transacao_local_com_impostos',
    'Taxa de serviço e tecnologia estimada (com tributos aplicáveis) na moeda local': 'taxa_servico_tecnologia_local',
    'Número da fatura': 'numero_fatura',
    'Deduções em moeda local': 'deducoes_local',
    'IsGroupOrder': 'eh_pedido_grupo',
    'Tipo de atendimento': 'tipo_atendimento',
    'País': 'pais',
    'Tipo de cancelamento': 'tipo_cancelamento',
    'Economia pela assinatura (moeda local)': 'economia_assinatura_local'
}


In [ ]:
# 3c. Renomeia as colunas conforme o dicionário
df = df.rename(columns=new_column_names)
df.head()

In [ ]:
# Passo 1: Listar os campos que você deseja alterar
colunas_para_alterar = ['hora_solicitacao_utc', 'hora_solicitacao_local', 'hora_chegada_utc', 'hora_chegada_local']

# Função para converter horários em objetos datetime.time
def converter_horario(horario):
    try:
        # Tenta interpretar o horário automaticamente
        return parser.parse(horario).time()
    except Exception:
        # Retorna NaT caso o valor seja inválido ou vazio
        return pd.NaT

# Passo 2: Aplicar a conversão nas colunas listadas diretamente no DataFrame original
df[colunas_para_alterar] = df[colunas_para_alterar].applymap(converter_horario)

# Exibir o DataFrame atualizado
df

In [ ]:
def convert_columns_to_iso_date(df: pd.DataFrame,
                                cols: Iterable[str],
                                inplace: bool = True,
                                as_string: bool = True,
                                infer_fallback: bool = True,
                                na_value: Optional[str] = None) -> Optional[pd.DataFrame]:
    """
    Converte colunas de df com datas no padrão MM/DD/YYYY (ou timestamps)
    para o padrão ISO YYYY-MM-DD.

    Parâmetros
    - df: DataFrame que contém as colunas.
    - cols: lista/iterável de nomes de colunas a converter.
    - inplace: se True altera df no lugar; se False retorna uma cópia modificada.
    - as_string: se True escreve strings 'YYYY-MM-DD'; se False deixa dtype datetime64[ns].
    - infer_fallback: se True, para valores não parseados com o formato exato
                      tenta pd.to_datetime (inferência) como fallback.
    - na_value: valor a usar quando a data é inválida e as_string=True (ex.: None, '', 'NA').
                Ignorado se as_string=False (fica NaT).
    Retorna
    - None se inplace=True; caso contrário retorna o DataFrame convertido.
    """
    if not inplace:
        df = df.copy()

    for c in cols:
        if c not in df.columns:
            print(f"[warn] coluna '{c}' não encontrada — pulando")
            continue

        ser = df[c]
        # Se já for datetime, usamos direto
        if pd.api.types.is_datetime64_any_dtype(ser):
            dt = ser.dt.normalize()  # manter só a data
        else:
            ser_str = ser.astype(str).str.strip().replace({'nan': None, 'None': None, '': None})

            # tentativa estrita para MM/DD/YYYY (mais rápida e determinística)
            dt = pd.to_datetime(ser_str, format="%m/%d/%Y", errors="coerce", dayfirst=False)

            # fallback permissivo apenas onde necessário
            if infer_fallback:
                mask_na = dt.isna() & ser_str.notna()
                if mask_na.any():
                    dt_fallback = pd.to_datetime(ser_str[mask_na], errors="coerce", infer_datetime_format=True, dayfirst=False)
                    dt.loc[mask_na] = dt_fallback

            # se houver strings como "2025-03-15 10:00:00" ou com timezone, o fallback captura
            # garantimos apenas a parte de data (normalizar)
            dt = dt.dt.normalize()

        # salvar no formato desejado
        if as_string:
            out = dt.dt.strftime("%Y-%m-%d")
            # strftime produz NaN para NaT; padronizamos para na_value (ou None)
            if na_value is None:
                out = out.where(~dt.isna(), None)
            else:
                out = out.where(~dt.isna(), na_value)
            df[c] = out
        else:
            df[c] = dt  # datetime64[ns]

        # resumo rápido por coluna
        n_total = len(df)
        n_valid = df[c].notna().sum() if as_string else df[c].notna().sum()
        n_invalid = n_total - n_valid
        print(f"[ok] coluna '{c}': total={n_total}, válidos={n_valid}, inválidos={n_invalid}")

    if not inplace:
        return df
    return None



In [ ]:
# ----------------- uso direto (executar abaixo desta célula) -----------------
# Exemplos de chamada (ajuste se quiser as strings ou datetime dtype):
# Converte as colunas indicadas em-place e grava strings 'YYYY-MM-DD' (None para inválidos)
# convert_columns_to_iso_date(df, ['data_solicitacao_utc','data_solicitacao_local','data_chegada_utc','data_chegada_local'],
#                             inplace=True, as_string=True, infer_fallback=True, na_value=None)

# Se preferir manter dtype datetime (recomendado para manipulações posteriores):
convert_columns_to_iso_date(df, ['data_solicitacao_utc','data_solicitacao_local','data_chegada_utc','data_chegada_local'], inplace=True, as_string=False)

# Obs: se quiser que a função retorne um novo DataFrame em vez de alterar o existente,
# use inplace=False e capture o retorno:
# df2 = convert_columns_to_iso_date(df, [...], inplace=False, as_string=True)

In [ ]:
df['data_hora_transacao_utc'] = pd.to_datetime(df['data_hora_transacao_utc'], errors='coerce')

In [ ]:
df

In [ ]:
cols = ['data_solicitacao_utc','data_solicitacao_local','data_chegada_utc','data_chegada_local']

# linhas onde qualquer coluna está NA/NaT
invalid_any = df[df[cols].isna().any(axis=1)].copy()

# resumo rápido
print("Total registros:", len(df))
print("Registros com qualquer data inválida:", len(invalid_any))
print("Contagem inválidos por coluna:")
print(df[cols].isna().sum())
# mostrar primeiras 10 linhas para inspeção
invalid_any.head(10)

In [ ]:
invalid_any.to_csv('erros_conversao_data.csv', sep=';', index=False)

In [ ]:
df = df.drop(index=invalid_any.index)
df.reset_index(drop=True, inplace=True)

In [ ]:
df_import = df[['nome_organizacao', 'id_viagem', 'data_hora_transacao_utc', 'data_solicitacao_utc', 'hora_solicitacao_utc', 'data_solicitacao_local', 'hora_solicitacao_local', 'data_chegada_utc', 'hora_chegada_utc', 'data_chegada_local', 'hora_chegada_local', 'nome', 'sobrenome', 'email', 'id_funcionario', 'servico', 'cidade', 'distancia_mi', 'duracao_min', 'endereco_partida', 'endereco_destino', 'codigo_despesa', 'detalhamento_despesa', 'programa', 'grupo','valor_transacao_local_com_tributos','valor_extra_local', 'deducoes_local']]

In [ ]:
df_import

In [ ]:
# Mapeamento de mês para abreviação em português
mes_abrev = {
    'January': 'jan', 'February': 'fev', 'March': 'mar', 'April': 'abr',
    'May': 'mai', 'June': 'jun', 'July': 'jul', 'August': 'ago',
    'September': 'set', 'October': 'out', 'November': 'nov', 'December': 'dez'
}

df_import['mes_solicitacao'] = df_import['data_solicitacao_local'].dt.month_name().map(mes_abrev)

# Move a coluna 'mes_solicitacao' para logo após 'data_solicitacao_local'
cols = list(df_import.columns)
cols.remove('mes_solicitacao')
idx = cols.index('data_solicitacao_local') + 1
cols = cols[:idx] + ['mes_solicitacao'] + cols[idx:]
df_import = df_import[cols]

In [ ]:
df_import['dia_da_semana_dt_solicitacao'] = df_import['data_solicitacao_local'].dt.day_name(locale='pt_BR')

cols = list(df_import.columns)
cols.remove('dia_da_semana_dt_solicitacao')
idx = cols.index('mes_solicitacao') + 1
cols = cols[:idx] + ['dia_da_semana_dt_solicitacao'] + cols[idx:]
df_import = df_import[cols]


In [ ]:
df_import.dtypes

In [ ]:
from datetime import time

def classificar_hora(hora_str):
    if pd.isna(hora_str):
        return None
    try:
        h, m, s = map(int, str(hora_str).split(':'))
        hora = time(h, m, s)
    except Exception:
        return None

    if time(8,0,0) <= hora <= time(12,0,0):
        return 'expediente_manha'
    elif time(12,1,0) <= hora <= time(14,0,0):
        return 'horario_almoco'
    elif time(14,1,0) <= hora <= time(18,0,0):
        return 'expediente_tarde'
    elif time(18,0,1) <= hora <= time(20,0,0):
        return 'entre 18h e 20h'
    elif time(20,0,1) <= hora <= time(23,59,0):
        return 'entre 20h e 23h59'
    elif (time(0,0,0) <= hora <= time(6,0,0)):
        return 'entre 00h e 06h'
    elif time(6,1,0) <= hora <= time(8,0,0):
        return 'entre 06h e 08h'
    else:
        return None

df_import['classif_hora_local'] = df_import['hora_solicitacao_local'].apply(classificar_hora)

In [ ]:
cols = list(df_import.columns)
cols.remove('classif_hora_local')
idx = cols.index('hora_solicitacao_local') + 1
cols = cols[:idx] + ['classif_hora_local'] + cols[idx:]
df_import = df_import[cols]

In [ ]:
# Converte colunas para float, tratando vírgula como separador decimal
for col in ['valor_transacao_local_com_tributos', 'valor_extra_local']:
    df_import[col] = (df_import[col].astype(str).str.replace(',', '.', regex=False).str.replace(r'[^0-9\.-]', '', regex=True).astype(float))

In [ ]:
# Remove caracteres não numéricos e ponto, substitui strings vazias ou '--' por NaN, converte para float e preenche nulos com 0
df_import['deducoes_local'] = (
    df_import['deducoes_local']
    .astype(str)
    .str.replace(r'[^0-9\.-]', '', regex=True)
    .replace(['', '--'], np.nan)
    .astype(float)
    .fillna(0)
    )

In [ ]:
df_import